In [1]:
import pandas as pd 
import numpy as np

# Statistial Analysis 

Topics:

 - Normalization (Min-Max)
 - Mean, Median, Mode
 - Standard Deviation + ±3σ rule (outlier idea)
 - Standardization (Z-score)
 - IQR (Interquartile Range) outliers
 - Skewness, Kurtosis

In [2]:
# We'll create a feature vector with a few outliers so we can see:
# - mean vs median difference
# - ±3σ and IQR outlier detection
# - skewness/kurtosis behavior

x = np.concatenate([
    np.random.normal(loc=50, scale=10, size=95),  # "typical" values
    np.array([120, 130, 140, 5, 8])               # outliers (high + low)
])

print("Count:", x.size)
print("First 10:", x[:10])

Count: 100
First 10: [41.35739588 43.82578231 52.20329995 60.92239801 42.10226469 52.0725408
 61.02316609 59.3541196  57.31396475 54.69059852]


In [12]:
from IPython.display import HTML

HTML("""
<iframe width="560" height="315" src="https://www.youtube.com/embed/i373-Vc2d4o?si=g33RiXfjTcZHKD5d" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>
""")


In [3]:
# Mean   = average; sensitive to outliers, so it doesn't give accurate analysis
# Median = middle value after sorting; robust to outliers
# Mode   = most frequent value; useful for categorical/discrete data

mean_x = np.mean(x)
median_x = np.median(x)

# Mode in pure NumPy (works best for discrete values)
vals, counts = np.unique(x, return_counts=True)
mode_x = vals[np.argmax(counts)]  # for continuous random data, mode often isn't meaningful

print("Mean  :", mean_x)
print("Median:", median_x)
print("Mode  :", mode_x, "(Note: mode is not very meaningful for continuous random data)")

Mean  : 52.03435801720113
Median: 51.96945485464805
Mode  : 5.0 (Note: mode is not very meaningful for continuous random data)


In [6]:
# Std dev (σ) measures typical spread around the mean.
# ±3σ rule (empirical rule): for roughly normal data, ~99.7% points lie within mean ± 3σ.
# So values outside mean ± 3σ are often treated as potential outliers.

std_x = np.std(x, ddof=0)  # population std (ddof=0). For sample std, use ddof=1.

lower_3s = mean_x - 3 * std_x
upper_3s = mean_x + 3 * std_x

outliers_3s = x[(x < lower_3s) | (x > upper_3s)]

print("Std dev:", std_x)
print("3σ range:", (lower_3s, upper_3s))
print("Outliers by ±3σ rule:", outliers_3s)
print("Outlier count:", outliers_3s.size)

Std dev: 17.88079379745161
3σ range: (-1.6080233751536994, 105.67673940955595)
Outliers by ±3σ rule: [120. 130. 140.]
Outlier count: 3


## Min–Max Normalization

$x_{\text{norm}} = \frac{x - x_{\min}}{x_{\max} - x_{\min}}$

## Standardization (Z-score)

$z = \frac{x - \mu}{\sigma}$

where $\mu$ is the mean and $\sigma$ is the standard deviation.

## Cons of Min–Max and how Standardization helps

**Cons of Min–Max**

Very sensitive to outliers: if one value is extremely large/small, $x_{\max}$ or $x_{\min}$ becomes extreme, and most normal values get squeezed into a tiny range near 0 or 1.

Range depends on the dataset: when new data arrives with a new max/min, the scaling changes, so the same value can map differently across datasets.

**How Standardization helps**

Less dominated by extreme min/max: it scales using $\mu$ and $\sigma$, so a single outlier usually affects scaling less than directly setting the entire range.

Produces comparable feature scales: features become centered at 0 with unit variance, which often helps gradient-based models (e.g., logistic regression, SVM, neural nets) learn faster and more stably.

In [7]:
# Normalization (Min-Max): rescales data into [0, 1]
# Formula: x_norm = (x - min(x)) / (max(x) - min(x))
# Useful when features have different units/ranges and you want them comparable.
# Note: very sensitive to outliers because min/max can be extreme.

x_min, x_max = x.min(), x.max()
x_norm = (x - x_min) / (x_max - x_min)

print("Min, Max:", x_min, x_max)
print("Normalized range:", (x_norm.min(), x_norm.max()))
print("First 5 normalized:", x_norm[:5])

Min, Max: 5.0 140.0
Normalized range: (0.0, 1.0)
First 5 normalized: [0.26931404 0.28759839 0.34965407 0.41423999 0.27483159]


In [8]:
# Standardization: makes data have mean ~0 and std ~1
# Formula: z = (x - mean) / std
# Helpful for models sensitive to scale (e.g., gradient descent, SVM, KNN, PCA).

x_z = (x - mean_x) / (std_x + 1e-8)  # epsilon avoids divide-by-zero

print("Standardized mean ~", x_z.mean())
print("Standardized std  ~", x_z.std())
print("First 5 standardized:", x_z[:5])

Standardized mean ~ 8.881784197001253e-17
Standardized std  ~ 0.9999999994407405
First 5 standardized: [-0.59711902 -0.45907222  0.00944823  0.49707189 -0.55546154]


In [9]:
# Quartiles:
# Q1 = 25th percentile
# Q2 = 50th percentile (median)
# Q3 = 75th percentile
#
# IQR = Q3 - Q1 (spread of middle 50% of data)
#
# IQR outlier rule (common robust rule):
# lower = Q1 - 1.5*IQR
# upper = Q3 + 1.5*IQR
# Values outside are potential outliers (more robust than ±3σ for skewed data).

Q1 = np.percentile(x, 25)
Q3 = np.percentile(x, 75)
IQR = Q3 - Q1

lower_iqr = Q1 - 1.5 * IQR
upper_iqr = Q3 + 1.5 * IQR

outliers_iqr = x[(x < lower_iqr) | (x > upper_iqr)]

print("Q1, Median, Q3:", Q1, median_x, Q3)
print("IQR:", IQR)
print("IQR range:", (lower_iqr, upper_iqr))
print("Outliers by IQR:", outliers_iqr)
print("Outlier count:", outliers_iqr.size)

Q1, Median, Q3: 42.41118769365814 51.96945485464805 57.95333626162045
IQR: 15.542148567962307
IQR range: (19.09796484171468, 81.26655911356391)
Outliers by IQR: [120. 130. 140.   5.   8.]
Outlier count: 5


In [10]:

# Skewness: measures asymmetry of distribution
#   skew > 0  => right-skewed (long right tail, many small values)
#   skew < 0  => left-skewed  (long left tail)
#   skew ~ 0  => symmetric
#
# Kurtosis: measures tail heaviness / peakedness (relative to normal)
# In pandas, kurtosis() returns "excess kurtosis":
#   ~0 => similar tails to normal
#   >0 => heavier tails (more outliers)
#   <0 => lighter tails

s = pd.Series(x)
skew = s.skew()
kurt = s.kurt()  # excess kurtosis

print("Skewness:", skew)
print("Kurtosis (excess):", kurt)

Skewness: 2.2176116625699014
Kurtosis (excess): 10.208477608360434


In [11]:
# When to use what (quick rules):
# - Mean/Std: good for roughly normal numeric features (but sensitive to outliers)
# - Median/IQR: robust summaries when outliers/skew exist
# - Min-Max normalization: when you want [0,1] scaling (but outlier-sensitive)
# - Z-score standardization: common default for many ML algorithms
# - ±3σ rule: simple outlier rule for near-normal data
# - IQR rule: robust outlier rule for skewed/non-normal data

summary = {
    "mean": mean_x,
    "median": median_x,
    "std": std_x,
    "min": x_min,
    "max": x_max,
    "Q1": Q1,
    "Q3": Q3,
    "IQR": IQR,
    "skewness": skew,
    "kurtosis_excess": kurt,
    "outliers_3sigma_count": int(outliers_3s.size),
    "outliers_iqr_count": int(outliers_iqr.size),
}
pd.Series(summary)

mean                      52.034358
median                    51.969455
std                       17.880794
min                        5.000000
max                      140.000000
Q1                        42.411188
Q3                        57.953336
IQR                       15.542149
skewness                   2.217612
kurtosis_excess           10.208478
outliers_3sigma_count      3.000000
outliers_iqr_count         5.000000
dtype: float64